---
title: "LS Bigdata School Homework 8"
author: iljoon
date: "2024-09-10"
categories: [homework]
jupyter: python3
---


# 로지스틱 회귀 예제

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

'''
종속변수: 백혈병 세포 관측 불가 여부 (REMISS), 1이면 관측 안됨을 의미

독립변수:

골수의 세포성 (CELL)
골수편의 백혈구 비율 (SMEAR)
골수의 백혈병 세포 침투 비율 (INFIL)
골수 백혈병 세포의 라벨링 인덱스 (LI)
말초혈액의 백혈병 세포 수 (BLAST)
치료 시작 전 최고 체온 (TEMP)
'''

# 문제 1. 데이터를 로드하고, 로지스틱 회귀모델을 적합하고, 회귀 표를 작성하세요.

In [ ]:
# 데이터 불러오기
df = pd.read_csv('../data/leukemia_remission.txt', sep='\t')
df

# 로지스틱 회귀모델 적합.
model = sm.formula.logit("REMISS ~ CELL + SMEAR + INFIL + LI + BLAST + TEMP", data=df).fit()

# 회귀 표 출력
print(model.summary())

# 문제 2. 해당 모델은 통계적으로 유의한가요? 그 이유를 검정통계량를 사용해서 설명하시오.

In [ ]:
'''
유의수준 5%일 때 유의하다.
LLR p-value: 0.04670 이기 때문에.
'''

# 문제 3. 유의수준이 0.2를 기준으로 통계적으로 유의한 변수는 몇개이며, 어느 변수 인가요?

In [ ]:
'''
유의수준 0.2 일 때 유의한 변수는 LI, TEMP
'''

# 문제 4. 다음 환자에 대한 오즈는 얼마인가요?

In [ ]:
# CELL (골수의 세포성): 65%
# SMEAR (골수편의 백혈구 비율): 45%
# INFIL (골수의 백혈병 세포 침투 비율): 55%
# LI (골수 백혈병 세포의 라벨링 인덱스): 1.2
# BLAST (말초혈액의 백혈병 세포 수): 1.1세포/μL
# TEMP (치료 시작 전 최고 체온): 0.9

## 로그 오즈 식
log_odds = 64.2581 + 30.8301 * 0.65 + 24.6863 * 0.45 -24.9745 * 0.55 + 4.3605 * 1.2 -0.0115 * 1.1 -100.1734 * 0.9
log_odds # -3.2655849999999873

## 오즈 식
my_odds = np.exp(log_odds)
my_odds # 0.03817459641135519

# 문제 5. 위 환자의 혈액에서 백혈병 세포가 관측되지 않은 확률은 얼마인가요?

In [ ]:
my_odds / (my_odds+1)
# 0.03677088280074742
# 3.6%

# 문제 6. TEMP 변수의 계수는 얼마이며, 해당 계수를 사용해서 TEMP 변수가 백혈병 치료에 대한 영향을 설명하시오.

In [ ]:
'''
TEMP가 0.01 오르면 로그 오즈가 -1.001734 만큼 증가한다.(사실 떨어지는거.)
이는 TEMP가 0.01 증가할 때마다 "백혈병 세포가 관측 되지 않는 것" 대한 오즈가 약 63.27% 떨어진다는 것이다.
'''
np.exp(-1.001734) # 0.3672420909627784
(0.3672420909627784 - 1) * 100

# 문제 7. CELL 변수의 99% 오즈비에 대한 신뢰구간을 구하시오.

In [ ]:
# 모르겠다!!

# 문제 8. 주어진 데이터에 대하여 로지스틱 회귀 모델의 예측 확률을 구한 후, 50% 이상인 경우 1로 처리하여, 혼동 행렬를 구하시오.

In [ ]:
df = df.assign(odds = 
    np.exp(
      64.2581 + 30.8301 * df['CELL'] + 24.6863 * df['SMEAR'] +
      (-24.9745) * df['INFIL'] + 4.3605 * df['LI'] +
      (-0.0115) * df['BLAST'] + (-100.1734) * df['TEMP']
    ))
df = df.assign(remiss_probability =
    df['odds'] / (df['odds'] + 1))
df = df.assign(classification =
    np.where(df['remiss_probability'] >= 0.5,
             1, 0))

# 문제 9. 해당 모델의 Accuracy는 얼마인가요?

In [ ]:
mymatrix = np.matrix([[
    ((df['classification'] == 1) & (df['REMISS'] == 1)).sum(),
    ((df['classification'] == 1) & (df['REMISS'] == 0)).sum()
    ],
    [
    ((df['classification'] == 0) & (df['REMISS'] == 1)).sum(),
    ((df['classification'] == 0) & (df['REMISS'] == 0)).sum()
    ]])

# 올바르게 예측한 샘플 개수
mymatrix[0, 0] + mymatrix[1, 1] # 20

# 전체 샘플 개수
mymatrix[0, 0] + mymatrix[0, 1] + mymatrix[1, 0] + mymatrix[1, 1] # 27

20/27 # Accuracy = 0.74

# 문제 10. 해당 모델의 F1 Score를 구하세요.

In [ ]:
precision = mymatrix[0, 0] / (mymatrix[0, 0] + mymatrix[0, 1]) # 0.625
recall = mymatrix[0, 0] / (mymatrix[0, 0] + mymatrix[1, 0]) # 0.555

2 * (precision * recall) / (precision + recall) # 0.588